In [19]:
import re

file_path = "/media/amhawk/1180b248-3121-46a2-892f-e237b8070717/Projects/logs/12_04_2023/07_12_06.log"
with open(file_path, "r") as f:
    log_data = f.read()

# Extract JSON structures using regex
json_pattern = re.compile(r'\{[^}]*\}', re.MULTILINE | re.DOTALL)
json_matches = re.finditer(json_pattern, log_data)

# Extract video IDs using regex
video_id_pattern = re.compile(r'video_id : (\d+)')
video_id_matches = re.finditer(video_id_pattern, log_data)

# Collect video IDs
video_ids = [match.group(1) for match in video_id_matches]

# Process JSON structures
json_quest = []
for match in json_matches:
    extracted_json = match.group()
    try:
        # Use eval cautiously, consider using json.loads for safer JSON parsing
        eval_json = eval(extracted_json)
        json_quest.append(eval_json)
    except Exception as e:
        print(f"Error parsing JSON: {e}")

# Combine video IDs and JSON structures into a list of dictionaries
result_list = [{'video_id': vid, 'questions': json} for vid, json in zip(video_ids, json_quest)]

# Create a DataFrame from the list of dictionaries
import pandas as pd
df = pd.DataFrame(result_list)

# Print the DataFrame
print(df)


Error parsing JSON: unterminated string literal (detected at line 3) (<string>, line 3)
     video_id                                          questions
0   698894249  {'question1': 'How would you ensure that the w...
1   698893976  {'question1': 'What are the different ways in ...
2   698893499  {'question1': 'How is Vijay related to Hari?',...
3   698488529  {'question1': 'What are the steps to create a ...
4   698488493  {'question1': 'What are the steps involved in ...
..        ...                                                ...
61  698477373  {'question1': 'What are the benefits of using ...
62  698477243  {'question1': 'What is the replay capability a...
63  698477192  {'question1': 'What are the benefits of using ...
64  698477136  {'question1': 'What are the benefits of using ...
65  698477046  {'question1': 'What is the difference between ...

[66 rows x 2 columns]


In [18]:
df.to_csv("./csv_output/ques_2100_2235.csv")

In [1]:
import ai21
import os
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

ai21.api_key = os.environ.get("AI21_API_KEY")

In [12]:
prompt = "What are the steps to create a new workflow in Alteryx to split the name column into two different columns?"
question_gen = ai21.Completion.execute(
        model='j2-ultra',
        prompt=prompt,
        numResults=1,
        maxTokens=1000,
        temperature=0.3,
        topKReturn=0,
        top_p=1,
        stopSequence=['##']
    )

In [13]:
generated_text = question_gen['completions'][0]['data']['text']
print(generated_text)


To create a new workflow in Alteryx to split the name column into two different columns, you can follow these steps:

1. Open Alteryx Designer and create a new workflow.
2. Drag and drop a "Data Input" tool onto the canvas.
3. Connect the "Data Input" tool to a "Data Output" tool.
4. In the "Data Input" tool, select the file or data source that contains the name column you want to split.
5. Drag and drop a "Split Column" tool onto the canvas.
6. Connect the output of the "Data Input" tool to the "Input" port of the "Split Column" tool.
7. Connect the "Output" port of the "Split Column" tool to a "Data Output" tool.
8. In the "Split Column" tool, select the "Name" column as the column to split.
9. In the "Split Column" tool, specify the separator or delimiter that you want to use to split the name column. For example, if you want to split the name column at the space character, you can use a space as the separator.
10. In the "Split Column" tool, specify the desired output columns for 

In [26]:
import pandas as pd

df = pd.read_csv("./ques_1700_1995_mods.csv")

In [4]:
x = df['questions']

In [9]:
x.head()

0    {'question1': 'What are the differences betwee...
1    {'question1': 'What is the difference between ...
2    {'question1': 'What are the different meta cha...
3    {'question1': 'What specific aspects of time s...
4    {'question1': 'What are the different steps in...
Name: questions, dtype: object

In [25]:
import ast

# Convert the data into a DataFrame
df = pd.DataFrame(x)

# Convert the 'questions' column from a string to a dictionary
df['questions'] = df['questions'].apply(ast.literal_eval)

# Expand the dictionary in the 'questions' column into separate columns
df = pd.concat([df.drop(['questions'], axis=1), df['questions'].apply(pd.Series)], axis=1)

# Save the DataFrame to a new CSV file
df.to_csv('output.csv', index=False)

print('Data has been written to output.csv.')

Data has been written to output.csv.
